In [ ]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
sequence_length = 16        # Number of frames in each sequence
learning_rate = 0.001
batch_size = 64
num_epochs = 10

# Get data
train_set = datasets.VideoClips(name='/content/drive/MyDrive/summ vid/videos',
                                transform=transforms.ToTensor(), 
                                sequence_length=sequence_length,
                                sample_every=4)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

test_set = datasets.VideoClips(name='/content/drive/MyDrive/summ vid/videos',
                               transform=transforms.ToTensor(), 
                               sequence_length=sequence_length,
                               sample_every=4)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False) 

# Build model
class VideoSummarizer(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv3d(3, 8, kernel_size=(3,3,3))
        self.maxpool1 = nn.MaxPool3d(kernel_size=(1,2,2))
        self.conv2 = nn.Conv3d(8, 16, kernel_size=(3,3,3))
        self.maxpool2 = nn.MaxPool3d(kernel_size=(1,2,2)) 
        self.fullyconn = nn.Linear(16*6*6*6, 2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x)) 
        x = self.maxpool1(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fullyconn(x)
        return x

model = VideoSummarizer().to(device)
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train 
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)
        
        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
    print('Epoch {}: Loss: {}'.format(epoch+1, loss.item()))

# Test 
model.eval() 
with torch.no_grad(): 
    correct = 0
    total = 0
    for data, targets in test_loader:
        data = data.to(device)
        targets = targets.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    print('Test Accuracy: {} %'.format(100 * correct / total))